# Association Rules Analysis
Gerçek hayattan bir süpermarkete ait veri seti ile gerçekleştirilmiştir.

In [1]:
# Gerekli import işlemleri gerçekleştirildi.
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
#Tüm sutünları ve satırları gözlemlemek için kullandığımız kod.
pd.set_option('display.max_columns', None); pd.set_option('display.max_rows', None);

In [3]:
# Veri seti okunuldu. Veri setinde sutünlarda değişkenler olmadığı için names argümanı ile değişkenlere isimler atandı.
df = pd.read_csv('../input/real-supermarket-data/satislar.csv',names = ['BranchId', 'PosId', 'InvoiceDate', 'InvoiceNo','StockCode','Line','Quantity','CategoryCode','CategoryName'],
                 low_memory=False,sep = ";", header = None)

In [4]:
# Veri setine ait ilk 5 gözleme erişildi.
df.head()

,BranchId,PosId,InvoiceDate,InvoiceNo,StockCode,Line,Quantity,CategoryCode,CategoryName
0,1145,2236,2020-05-01 08:29:49.000,11450001261903,426410,0,1,05.07.014,"Tıraş Bıçak ve Jiletleri,,"
1,1129,2156,2020-05-01 08:30:44.000,11290001181201,59010,0,"1,5",03.03.002,"Dana Eti Kg,"
2,1141,2216,2020-05-01 08:36:56.000,11410001201443,314720,0,1,02.02.004,"Meyve Suları Küçük Boy,,"
3,1141,2216,2020-05-01 08:36:56.000,11410001201443,180122,1,1,01.07.002,"Çikolata,,"
4,1141,2216,2020-05-01 08:36:56.000,11410001201443,162169,2,1,01.04.002,"Bisküvi Çeşitleri,,"


In [5]:
# Veri setinin boyutuna erişildi.
df.shape

(1048576, 9)

In [6]:
# Veri setinde eksik gözlem var mı sorusuna yanıt alındı.
df.isnull().sum()

BranchId            0
PosId               0
InvoiceDate         0
InvoiceNo           0
StockCode           0
Line                0
Quantity            0
CategoryCode    73833
CategoryName        0
dtype: int64

In [7]:
# CategoryCode ile birliktelik analizi yapmayacağız için silebiliriz.
df.dropna(subset= ["CategoryCode"],inplace= True)
df.shape

(974743, 9)

In [8]:
# Veri setinde eksik gözlem olup olmadığı kontrol edildi.
df.isnull().sum()

BranchId        0
PosId           0
InvoiceDate     0
InvoiceNo       0
StockCode       0
Line            0
Quantity        0
CategoryCode    0
CategoryName    0
dtype: int64

In [9]:
# Veri setinde her değişken için eşsiz gözlem sayısı kaç tane buna erişildi.
df.nunique()

BranchId            55
PosId               83
InvoiceDate     194210
InvoiceNo       256495
StockCode         6131
Line                97
Quantity          4521
CategoryCode       258
CategoryName       277
dtype: int64

In [10]:
#hangi üründen kaçar tane var?
df["CategoryName"].value_counts().head()

Sebze,                 151138
Çikolata,,              51261
Sigara,,                48101
Meyve,                  35987
Bisküvi Çeşitleri,,     34495
Name: CategoryName, dtype: int64

In [11]:
# Veri setinin betimsel istatistiklerine erişim sağlandı.
df.describe().T

,count,mean,std,min,25%,50%,75%,max
BranchId,974743.0,1.535722e+03,1.004966e+03,1.101000e+03,1.117000e+03,1.131000e+03,1.144000e+03,4.010000e+03
PosId,974743.0,2.454675e+03,6.997097e+02,2.016000e+03,2.097000e+03,2.167000e+03,2.236000e+03,4.021000e+03
InvoiceNo,974743.0,1.535723e+13,1.004966e+13,1.101000e+13,1.117000e+13,1.131000e+13,1.144000e+13,4.010000e+13
StockCode,974743.0,2.065353e+05,1.606901e+05,1.011000e+04,1.260050e+05,1.543620e+05,3.141530e+05,9.000674e+06
Line,974743.0,3.802201e+00,5.505346e+00,0.000000e+00,0.000000e+00,2.000000e+00,5.000000e+00,9.600000e+01


In [12]:
# Veri setindeki CategoryName değişkeninde yer alan ürünleri strip komutu ile ayırdık.
df['CategoryName'] = df['CategoryName'].str.strip(',')
df.head()

,BranchId,PosId,InvoiceDate,InvoiceNo,StockCode,Line,Quantity,CategoryCode,CategoryName
0,1145,2236,2020-05-01 08:29:49.000,11450001261903,426410,0,1,05.07.014,Tıraş Bıçak ve Jiletleri
1,1129,2156,2020-05-01 08:30:44.000,11290001181201,59010,0,"1,5",03.03.002,Dana Eti Kg
2,1141,2216,2020-05-01 08:36:56.000,11410001201443,314720,0,1,02.02.004,Meyve Suları Küçük Boy
3,1141,2216,2020-05-01 08:36:56.000,11410001201443,180122,1,1,01.07.002,Çikolata
4,1141,2216,2020-05-01 08:36:56.000,11410001201443,162169,2,1,01.04.002,Bisküvi Çeşitleri


In [13]:
#en çok sipariş edilen ürün hangisi?
df.groupby("CategoryName").agg({"Quantity":"sum"}).sort_values("Quantity", ascending = False).head()

,Quantity
CategoryName,
Devir Urunleri Yummy,7111111111111111141111111111111111111111114611...
Mangal Malzemeleri,4111111111111112111221132114111211111211141113...
Dondurma,4111111111111111111111111111111311111111111111...
Mum,3021111111143122611115111112111211121138413313...
Gazete & Dergi,22111111111


In [14]:
# Veri setinde yer alan CategoryName değişkenine ait normalize ederek frekans değerleri getirilmiştir.
pd.DataFrame(df["CategoryName"].value_counts(normalize=True)).head(100)

,CategoryName
Sebze,0.188234
Çikolata,0.052589
Sigara,0.049347
Meyve,0.037109
Bisküvi Çeşitleri,0.035389
Ekmek,0.031891
Dana Eti Kg,0.028740
Kola,0.028221
Piliç Eti Kg,0.027169
Cips,0.022813


In [15]:
#Veri setinde yer alan Quantity değişkenini integera çevirmekteyiz. 
df['Quantity'] = [x.replace(',', '.') for x in df['Quantity']]
df["Quantity"] = df["Quantity"].astype("float")
df["Quantity"] = df["Quantity"].astype("int")

In [16]:
#Veri setinin yapısal bilgilerine erişim sağlandı aynı zamanda Quantity değişkeninin integer olduğunu gözlemlemekteyiz.
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 974743 entries, 0 to 1048575
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   BranchId      974743 non-null  int64 
 1   PosId         974743 non-null  int64 
 2   InvoiceDate   974743 non-null  object
 3   InvoiceNo     974743 non-null  int64 
 4   StockCode     974743 non-null  int64 
 5   Line          974743 non-null  int64 
 6   Quantity      974743 non-null  int64 
 7   CategoryCode  974743 non-null  object
 8   CategoryName  974743 non-null  object
dtypes: int64(6), object(3)
memory usage: 74.4+ MB


In [17]:
branch_order = (df.groupby(['InvoiceNo', 'CategoryName'])['Quantity'] .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo'))

In [18]:
branch_order.head()

CategoryName,Agız Bakım Suyu,Ahsap Temizleyici,Ahsap Urunler,Aluminyum Folyo,Ampul Cesitleri,Arap Sabunu,Ayakkabı Boya Malzemeleri,Ayran,Ayçiçek Yağı,Ağda Malzemeleri,Baharat Paketli,Baharat Set,Bal,Balık Konserveleri,Banyo Sabunları,Barbunya,Bayram Sekeri,Bebe Biberon ve Emzikleri,Bebek Bakım Urunleri,Bebek Bisküvi ve Ekmeği,Bebek Deterjanları,Bebek Sabun ve Şampuanları,Biber Salçası,Bijuteri Cesitleri,Bisküvi Çeşitleri,Bocek Oldurucu,Bulaşık Eldiveni,Bulaşık Jel Yıkama Deterjan,Bulaşık Makinesi Makine Temizleyici,Bulaşık Makinesi Parlatıcı,Bulaşık Makinesi Sıvı Deterjanı,Bulaşık Makinesi Tablet Deterjan,Bulaşık Makinesi Toz Deterjan,Bulaşık Makinesi Tuz,Bulaşık Sungeri ve Fırcası,Bulaşık Sıvı Yıkama Deterjanı,Bulgur,Bulyon,Buzdolabı Poseti,Buğday,Cam Urunler,"Cam, Parlak Yüzey Temizleyici",Cemen & Kahvaltılık Sos,Cikolata Sosu,Cips,Cop Poseti,Dana Eti Kg,Devir Urunleri Akide,Devir Urunleri Elazıg,Devir Urunleri Kakakocan,Devir Urunleri Kovancilar,Devir Urunleri Yummy,Dis Ipi ve Kurdanlar,Dis Parlatıcıları,Diş Fırçaları,Diş Macunları,Domates Salçası,Dondurma,Donuk Et Urunleri,Donuk Hamur İsleri,Donuk Sebze Urunleri,Dus Jeli,Ekmek,Eksiler(Nar),El ve Yuz Kremleri,El ve Yüz Katı Sabunlar,El ve Yüz Sıvı Sabunlar,Elde Yıkama Deterjanı,Eldiven Cok Amaclı,Enerji İçeceği,Ev Tekstili Urunleri,"Faras, Fırca, Supurge",Fasulye,Fındık Yagı,Fırca ve Taraklar,Galeta&Grisini,Gazete & Dergi,Gazoz,Genel Kireç Çözücü,Genel Leke Çıkarıcı ve Koruyucular,Gofret,Harçlar,Hasta Bezi,Hediyelik Cikolata,Helva,Helva Acık,Hırdavat Muhtelif,Islak Mendil,Kadın Bagları,Kahvaltılık Gevrek,Kahve Hazır,Kahve Klasik,Kakao,Kapı ve Pencere Bandı Cesitleri,Karışım Salça,Kavurma,Kaymak,Kağıt Havlu,Kefir,Kek ve Un Karısımları,Kek ve Pasta,Ketçap,Kibrit ve Cakmak Cesitleri,Kisisel Tekstil Urunleri,Kola,Kolonya ve Gülsuyu Çeşitleri,Kraker,Krem Fındık Ezmesi,Krem Santi & Pasta Kreması,Krem Çikolata,Krema,Kup Seker,Kurdan,Kuru Tatlılar,Kuruyemis Paketli,Kuruyemiş Dökme,Kuzu Eti Kg,Kırtasiye,Lavabo & Wc Kokuları,Lavabo Açıcıları,Leke Cıkarıcı,Limon Sosu,Limonata,Lokum,Makarna,Mama Kavanoz,Mama Toz,Mangal Malzemeleri,Mantı,Margarin,Matik Sıvı Deterjan,Matik Toz Deterjan,Maya,Mayonez,Mercimek,Metal Urunler,Meyve,Meyve Kurusu,Meyve Suları Küçük Boy,Meyve Suları Litre,Meyveli Toz Icecek,Muftelif Zuccaciye,Muhtelif Kozmetik,Muhtelif Kırtasiye Malzemeleri,Mum,Mutfak Yardımcı Malzemeleri,Mutfak ve Banyo Temizleyici,Mısır,Mısırözü Yağı,Nişasta,Nohut,Oda Kokusu,Oyuncak Cesitleri,Pamuk ve Kulak Pamugu Cesitleri,Parfum ve Deodorantlar,Paspas & Mop,Pasta Altı Kek,Pasta Kuru,Pasta Yardımcı Malzemeleri,Pasta Yas,Pastırma,Pekmez,Peynir Dökme,Peynir Kasar,Peynir Krem,Peynir Paketli,Peynir Tulum,Peçete,Pil Cesitleri,Piliç Eti Kg,Pirinç,Pirinç Unu,"Pisirme Torbası, Kagıdı",Pismaniye,Plastik Urunler,Porselen Urunler,Pudra Şekeri,Reçel,Rolon & Sticler,Sac Boya ve Kınaları,Sac Jole ve Spreyleri,Sac Kremi,Sakız,Salam,Sarf Urunler Et,Sebze,Sebze Konservesi,Sekerleme,Sigara,Silgi ve Kalemtraslar,Sirke Elma,Sirke Uzum,Soda-Maden Suyu,Sos Acı,Sos Tatlı,Sosis,Soğuk Çay,Strec Film,Su,Sucuk,Sut Meyveli,Sut Sade,Sut Tozu ve Krema,Sutlu Tatlılar,Sıcak Çikolata & Salep,Tahin,Tahin & Pekmez Karısımı,Tatlı,Telefon Aksesuarı,"Temizlik Bezi, Sungeri",Temizlik Setleri,Tereyağı,Tereyağı Açık,Terlik ve Ayakkabı Cesitleri,Toz Puding,Toz Sabun,Toz Seker,Toz Sütlü Tatlılar,Tras Kopuk ve Kremleri,Tras Sonrası Bakım Urunleri,Tul ve Halı Yıkama,Tulum Peyniri Dokme,Tursu Paketli,Tuvalet Kağıdı,Tuz,Tuz Ruhu,Tıraş Bıçak ve Jiletleri,Un,Un Galeta,Un Mısır,Vanilya & Kabartma Tozu,Yemek Konservesi,Yoğurt,Yoğurt Meyveli,Yufka,Yumurta,Yumuşatıcı,Yüzey Temizleyici,Zeytin Dokme Yesil,Zeytin Dökme Siyah,Zeytin Paketli Siyah,Zeytin Paketli Yeşil,Zeytinyağı,Çamaşır Kireç Önleyici,Çamaşır Sodası,Çamaşır Suyu,Çay Aromaları,Çay Bardak,Çay Bitki,Çay Demlik,Çay Kg,Çikolata,Çocuk Bezi,Çorba,İrmik,Şalgam Suyu,Şampuan
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [19]:
# Veri setini 1 ve 0 hale dönüştürme. One Hot Encoding işlemi yapılmıştır.
branch_encoded = branch_order.applymap(lambda x: 0 if x<=0 else 1) 
basket_branch = branch_encoded 

In [20]:
#Apriori en yaygın kullanılan fonk olarak da bilinmektedir. Support değeri için bir eşik değeri seçilir ve support değeri hesaplanır.
frq_items = apriori(basket_branch, min_support = 0.01, use_colnames = True)

In [21]:
# Support işleminin ilk beş gözlem birimine erişilmiştir.
frq_items.head()

,support,itemsets
0,0.010608,(Ayran)
1,0.017182,(Ayçiçek Yağı)
2,0.018928,(Baharat Paketli)
3,0.067303,(Bisküvi Çeşitleri)
4,0.017248,(Bulaşık Sıvı Yıkama Deterjanı)


In [22]:
#association_rules fonk kullanılarak "confidence ve lift" değerlerimize erişmiş olduk. Peki bu değerler neyi ifade etmektedir.
#Support: Örneğin X ve Y ürünleri olsun. Bu ikisinin birlikte görülme olasılığını ifade etmektedir.
#Confidence: Örneğin X ve Y ürünleri olsun. X'i alanların % şu kadarı Y'yide alacaklar. Bu bize confidence değerini verecektir.
#Lift: Bir ürünün alınması diğer ürünün alınmasını % kaç arttırmaktadır. Bizlere bu bilgiyi vermektedir. 
#Bu bizim için aynı zamanda final tablosudur.
rules = association_rules(frq_items, metric ="confidence", min_threshold = 0.20) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head(50) 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(Kraker),(Bisküvi Çeşitleri),0.021326,0.067303,0.011353,0.532358,7.909821,0.009918,1.994468
12,(Meyve),(Sebze),0.048157,0.230854,0.021178,0.439767,1.904953,0.010061,1.372903
1,(Kek ve Pasta),(Bisküvi Çeşitleri),0.035178,0.067303,0.014460,0.411061,6.107571,0.012093,1.583689
9,(Kek ve Pasta),(Çikolata),0.035178,0.091249,0.012636,0.359193,3.936392,0.009426,1.418135
5,(Bisküvi Çeşitleri),(Çikolata),0.067303,0.091249,0.021533,0.319933,3.506138,0.015391,1.336266
7,(Dana Eti Kg),(Sebze),0.040480,0.230854,0.012760,0.315227,1.365479,0.003415,1.123212
15,(Yumurta),(Sebze),0.042075,0.230854,0.012924,0.307172,1.330587,0.003211,1.110154
6,(Cips),(Çikolata),0.044964,0.091249,0.012647,0.281280,3.082541,0.008544,1.264401
14,(Sut Sade),(Sebze),0.055147,0.230854,0.014983,0.271686,1.176872,0.002252,1.056063
13,(Piliç Eti Kg),(Sebze),0.046820,0.230854,0.012534,0.267716,1.159674,0.001726,1.050338


# YORUMLAR
**Birliktelik analizi için gerekli olan değerlerimizi (support, confidence, lift) hesapladık ve buradan 3 çift seçilerek yorumlar yapılacaktır. Bu yorumlar için çiftler aşağıda yer almaktadır.**
- İlk olarak Kraker ve Bisküvi Çeşitleri  ele alınacaktır. 

- Daha sonra Meyve ve Sebze ele alınacaktır. 

- Son olarak (Kek ve Pasta) ve Bisküvi Çeşitleri ele alınacaktır. 

**1)     Kraker ve Bisküvi Çeşitleri ele alındığında;**
   - Kraker ve Bisküvi tüm alışverişlerin %1.13'ünde birlikte gözlenmektedir. Yani 100 alışverişin 1.13'ünde Kraker ve Bisküvi beraber alınmaktadır.

   - Kraker alan müşterilerin %53'ü Bisküvi'de almaktadır. Yani 100 kişi Kraker alıyorsa 53 kişi Kraker'in yanına Bisküvi'de almaktadır. 

   - Kraker olan alışverişlerde Bisküvi ürünün satışı 7,90 kat artmaktadır. 

**2)     Meyve ve Sebze ele alındığında;**
   - Meyve ve Sebze tüm alışverişlerin %2.11'inde birlikte gözlenmektedir. Yani 100 alışverişin 2.11'inde Meyve ve Sebze beraber alınmaktadır. 

   - Meyve alan müşterilerin %43'ü de Sebze'de almaktadır. Yani 100 kişi Meyve alıyorsa 43 kişi Meyve'nin yanına Sebze'de almaktadır. 

   - Meyve olan alışverişlerde Sebze ürünün satışı 1,90 kat artmaktadır. 
  
**3)     (Kek ve Pasta) ve Bisküvi Çeşitleri ele alındığında;**
   - (Kek ve Pasta) ve Bisküvi tüm alışverişlerin %1,44'ünde birlilkte gözlenmektedir. Yani 100 alışverişin 1.44'ünde (Kek ve Pasta) ve Bisküvi beraber alınmaktadır.

   - (Kek ve Pasta) alan müşterilerin %41'i Bisküvi'de almaktadır. Yani 100 kişi (Kek ve Pasta) alıyorsa 41 kişi (Kek ve Pasta)'nın yanına Bisküvi'de almaktadır. 

   - (Kek ve Pasta) olan alışverişlerde Bisküvi ürünün satışı 6,10 kat artmaktadır.
   